# Feature postprocessing from paper.
- Title:  "Predicting yeast synthetic lethal genetic interactions using protein domains"
- Authors: Bo Li, Feng Luo,School of Computing,Clemson University,Clemson, SC, USA
- e-mail: bol, luofeng@clemson.edu
- year:2009

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict 
import seaborn as sns
import matplotlib.cm as cm
import scipy as scipy
import random

### Importing datasets 

#### Link to the github repo where the datasets to be downloaded:
[DOWNLOAD THE DATASETS HERE](https://github.com/leilaicruz/machine-learning-for-yeast/tree/dev_Leila/datasets-for-learning)

In [2]:
import os
script_dir = os.path.dirname('__file__') #<-- absolute dir the script is in
rel_path_SL = "datasets/data-synthetic-lethals.xlsx"
rel_path_nSL="datasets/data-positive-genetic.xlsx"
rel_path_domains="datasets/proteins-domains-from-Pfam.xlsx"

abs_file_path_SL = os.path.join(script_dir, rel_path_SL)
abs_file_path_nSL = os.path.join(script_dir, rel_path_nSL)
abs_file_path_domains = os.path.join(script_dir, rel_path_domains)

# os.chdir('mini_book/docs/') #<-- for binder os.chdir('../')
# os.chdir('../')
my_path_sl= abs_file_path_SL
my_path_non_sl=abs_file_path_nSL
my_path_domains=abs_file_path_domains

data_sl=pd.read_excel(my_path_sl,header=0)
data_domains=pd.read_excel(my_path_domains,header=0,index_col='Unnamed: 0')
data_domains=data_domains.dropna()
data_nonsl=pd.read_excel(my_path_non_sl,header=0)

## Building the feature matrix
One matrix for true SL where each row is one pair of SL. Every raw will be a vector of 0,1 or 2 depending on the comparison with the domain list. For row i the jth element = 0 if the jth element of the domain list is not in neither protein A and B, 1, if it is in one of them and 2 if it is in both of them .

### Building the list of proteins domains id per protein pair separately :
- List of protein A: Search for the Sl/nSL database the query gene name and look in the protein domain database which protein domains id has each of those queries.
- List of protein B: Search for the Sl/nSL database the target gene name of the previous query and look in the protein domain database which protein domains id has each of those target genes.

In [3]:
# Selecting the meaningful columns in the respective dataset
domain_id_list=data_domains['domain-name']
query_gene=data_sl['gene-query-name']
target_gene=data_sl['gene-target-name']
query_gene_nonlethal=data_nonsl['gene-query-name']
target_gene_nonlethal=data_nonsl['gene-target-name']



# Initialising the arrays
protein_a_list=[]
protein_b_list=[]
protein_a_list_non=[]
protein_b_list_non=[]

population = np.arange(0,len(data_sl))

# For loop for 10000 pairs sampled randomly from the SL/nSl pair list , and creating a big array of proteind domains id per protein pair

for m in random.sample(list(population), 500):
    protein_a=data_domains[data_domains['name']==query_gene[m]]
    protein_b=data_domains[data_domains['name']==target_gene[m]]
    protein_a_list.append(protein_a['domain-name'].tolist())
    protein_b_list.append(protein_b['domain-name'].tolist())

    protein_a_non=data_domains[data_domains['name']==query_gene_nonlethal[m]]
    protein_b_non=data_domains[data_domains['name']==target_gene_nonlethal[m]]
    protein_a_list_non.append(protein_a_non['domain-name'].tolist())
    protein_b_list_non.append(protein_b_non['domain-name'].tolist())

In [4]:
print('We are going to analyze',len((protein_a_list)) ,'protein pairs, out of',len(data_sl),'SL protein pairs')
print('We are going to analyze',len((protein_a_list_non)) ,'protein pairs, out of',len(data_nonsl),'positive protein pairs')

We are going to analyze 500 protein pairs, out of 17871 SL protein pairs
We are going to analyze 500 protein pairs, out of 43340 positive protein pairs


### Postprocessing #1:  Remove protein pairs from study if either protein in the pair does not contain any domain

In [5]:
def remove_empty_domains(protein_list_search,protein_list_pair):
    index=[]
    for i in np.arange(0,len(protein_list_search)):
        if protein_list_search[i]==[] or protein_list_pair[i]==[]:
            index.append(i) ## index of empty values for the protein_a_list meaning they dont have any annotated domain

    y=[x for x in np.arange(0,len(protein_list_search)) if x not in index] # a list with non empty values from protein_a list

    protein_list_search_new=[]
    protein_list_pair_new=[]
    for i in y:
        protein_list_search_new.append(protein_list_search[i])
        protein_list_pair_new.append(protein_list_pair[i])
    return protein_list_search_new,protein_list_pair_new

## evaluating the function

protein_a_list_new,protein_b_list_new=remove_empty_domains(protein_a_list,protein_b_list)

protein_a_list_non_new,protein_b_list_non_new=remove_empty_domains(protein_a_list_non,protein_b_list_non)

In [6]:
print('The empty domain in the SL were:', len(protein_a_list)-len(protein_a_list_new), 'out of', len(protein_a_list),'domains')
print('The empty domain in the nSL were:', len(protein_a_list_non)-len(protein_a_list_non_new), 'out of', len(protein_a_list_non),'domains')

The empty domain in the SL were: 84 out of 500 domains
The empty domain in the nSL were: 73 out of 500 domains


### Feature engineering: Select from each ordered indexes of domain id list which of them appear once, in both or in any of the domains of each protein pair

#### Define function `get_indexes`

In [7]:
get_indexes = lambda x, xs: [i for (y, i) in zip(xs, range(len(xs))) if x == y] # a function that give the index of whether a value appear in array or not
a=[1,2,2,4,5,6,7,8,9,10]
get_indexes(2,a)

[1, 2]

In [8]:
def feature_building(protein_a_list_new,protein_b_list_new):
    x = np.unique(domain_id_list)
    ## To avoid taking repeated domains from one protein of the pairs , lets reduced the domains of each protein from the pairs to their unique members
    protein_a_list_unique=[]
    protein_b_list_unique=[]
    for i in np.arange(0,len(protein_a_list_new)):
        protein_a_list_unique.append(np.unique(protein_a_list_new[i]))
        protein_b_list_unique.append(np.unique(protein_b_list_new[i]))
        
    protein_feat_true=np.zeros(shape=(len(x),len(protein_a_list_unique)))
    pair_a_b_array=[]
    for i in np.arange(0,len(protein_a_list_unique)):
        index_a=[]
        
        pair=[protein_a_list_unique[i],protein_b_list_unique[i]]
        pair_a_b=np.concatenate(pair).ravel()
        pair_a_b_array.append(pair_a_b)

    j=0
    for i in pair_a_b_array:  
        array,index,counts=np.unique(i,return_index=True,return_counts=True)
        
        for k,m in zip(counts,array):
            if k ==2:
                protein_feat_true[get_indexes(m,x),j]=2
                
            if k==1:
                protein_feat_true[get_indexes(m,x),j]=1
        j=j+1
    return protein_feat_true

In [9]:
protein_feat_true=feature_building(protein_b_list_new=protein_b_list_new,protein_a_list_new=protein_a_list_new)
protein_feat_true_pd=pd.DataFrame(protein_feat_true.T)

protein_feat_non_true=feature_building(protein_b_list_new=protein_b_list_non_new,protein_a_list_new=protein_a_list_non_new)
protein_feat_non_true_pd=pd.DataFrame(protein_feat_non_true.T)

### How many ones and twos are in each dataset

In [10]:
index_2_true=protein_feat_true_pd.where(protein_feat_true_pd==2)
index_2_true_count=index_2_true.count(axis=1).sum()

index_1_true=protein_feat_true_pd.where(protein_feat_true_pd==1)
index_1_true_count=index_1_true.count(axis=1).sum()

index_2_nontrue=protein_feat_non_true_pd.where(protein_feat_non_true_pd==2)
index_2_nontrue_count=index_2_nontrue.count(axis=1).sum()

index_1_nontrue=protein_feat_non_true_pd.where(protein_feat_non_true_pd==1)
index_1_nontrue_count=index_1_nontrue.count(axis=1).sum()


print('fraction of twos in the SL array is',index_2_true_count/(len(protein_feat_true_pd.index)*len(protein_feat_true_pd.columns)))
print('fraction of ones in the SL array is',index_1_true_count/(len(protein_feat_true_pd.index)*len(protein_feat_true_pd.columns)))
print('fraction of twos in the PI array is',index_2_nontrue_count/(len(protein_feat_non_true_pd.index)*len(protein_feat_non_true_pd.columns)))
print('fraction of ones in the PI array is',index_1_nontrue_count/(len(protein_feat_non_true_pd.index)*len(protein_feat_non_true_pd.columns)))

fraction of twos in the SL array is 3.0197075651621106e-05
fraction of ones in the SL array is 0.0009710743801652893
fraction of twos in the PI array is 8.516074089844582e-06
fraction of ones in the PI array is 0.0008771556312539919


#### Bar plot to visualize these numbers

In [11]:
plt.bar(['fraction of 2 in the nSL','fraction of 1 in the nSL'],[index_2_nontrue_count/(len(protein_feat_non_true_pd.index)*len(protein_feat_non_true_pd.columns)),index_1_nontrue_count/(len(protein_feat_non_true_pd.index)*len(protein_feat_non_true_pd.columns))],alpha=0.6,color=['blue','lightblue']), 

plt.bar(['fraction of 2 in SL ','fraction of 1 in SL'],[index_2_true_count/(len(protein_feat_true_pd.index)*len(protein_feat_true_pd.columns)),index_1_true_count/(len(protein_feat_true_pd.index)*len(protein_feat_true_pd.columns))],alpha=0.6,color=['coral','lightcoral'])

plt.ylabel('Fraction from the population')
plt.yscale('log')
plt.xticks(rotation=40)

([0, 1, 2, 3], <a list of 4 Text major ticklabel objects>)

#### Adding the labels(response variables) to each dataset

In [12]:
protein_feat_true_pd['lethality']=np.ones(shape=(len(protein_a_list_new)))
protein_feat_non_true_pd['lethality']=np.zeros(shape=(len(protein_a_list_non_new)))

#### Joining both datasets

In [13]:
feature_post=pd.concat([protein_feat_true_pd,protein_feat_non_true_pd],axis=0)
feature_post=feature_post.set_index(np.arange(0,len(protein_a_list_new)+len(protein_a_list_non_new)))
print('The number of features are:',feature_post.shape[1])
print('The number of samples are:',feature_post.shape[0])

The number of features are: 3026
The number of samples are: 843


### Postprocessing and exploration of the feature matrix of both datasets

In [14]:
mean=feature_post.T.describe().loc['mean']
std=feature_post.T.describe().loc['std']
lethality=feature_post['lethality']

corr_keys=pd.concat([mean,std,lethality],axis=1)

### Viz of the stats

In [15]:
pair=sns.pairplot(corr_keys,hue='lethality',diag_kind='kde',kind='reg',palette='colorblind')
pair.fig.suptitle('Pairplot to see data dependencies with Lethality',y=1.08)

Text(0.5, 1.08, 'Pairplot to see data dependencies with Lethality')

In [16]:
X, y = feature_post.drop(columns=["lethality"]), feature_post["lethality"]

In [17]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_sl=protein_feat_true_pd.drop(columns=['lethality'])
x_nsl=protein_feat_non_true_pd.drop(columns=['lethality'])

X_sl_scaled = scaler.fit_transform(x_sl)
X_nsl_scaled = scaler.fit_transform(x_nsl)

In [18]:
np.shape(X_sl_scaled)

(416, 3025)

## How redundant are each of the protein domains?

In [19]:
def PCA_component_contribution(scaled_matrix,original_data):
    from sklearn.decomposition import PCA

    model = PCA(0.95).fit(scaled_matrix)
    ## apply dimensionality reduction to X_train
    output_pca = model.transform(scaled_matrix)
    
    total=sum(model.explained_variance_)


   
    # number of components , that it will be the number of main axes times the number of original components 
    n_pcs= model.components_.shape[0] # the amount of non redundant protein domains 

    # get the index of the most important feature on EACH component
    # LIST COMPREHENSION HERE
    most_important = [np.abs(model.components_[i]).argmax() for i in range(n_pcs)]

    initial_feature_names = original_data.columns
    # get the names
    most_important_names = [initial_feature_names[most_important[i]] for i in range(n_pcs)]
    

    # LIST COMPREHENSION HERE AGAIN
    dic = {'PC{}'.format(i): most_important_names[i] for i in range(n_pcs)}

    # build the dataframe
    df = pd.DataFrame(dic.items(),columns=['pca-component','domain-number'])

    return df,model.components_

In [20]:
df_sl,components_pca_nsl=PCA_component_contribution(X_nsl_scaled,x_nsl)
df_nsl,components_pca_sl=PCA_component_contribution(X_sl_scaled,x_sl)

In [21]:
df_sl.head()

pca-component  domain-number
0           PC0            242
1           PC1            191
2           PC2            352
3           PC3             58
4           PC4              3

## which are the shared domains between nSL and SL pairs , in the PCA reduced samples

In [22]:
df_shared_domains=pd.merge(df_sl,df_nsl,how='inner',on='domain-number')
domains_name=np.unique(data_domains['domain-name'])
domains_description=np.unique(data_domains['domain-descrip'])

df_shared_domains['domain-description']=domains_description[df_shared_domains['domain-number']]


df_shared_domains

pca-component_x  domain-number pca-component_y  \
0              PC2            352            PC16   
1              PC7            926            PC30   
2             PC16            100            PC41   
3             PC20            274             PC3   
4             PC21            876            PC87   
5             PC31            488            PC14   
6             PC40             36            PC28   
7             PC47           1072            PC66   
8             PC52            754            PC35   
9             PC53             38           PC195   
10            PC58           2710            PC80   
11            PC58           2710            PC81   
12            PC59            450           PC113   
13            PC66           1101            PC57   
14            PC66           1101            PC59   
15            PC67            440            PC74   
16            PC68            214            PC19   
17            PC74           1021           PC154   
18            PC79            773            PC49   
19            PC81            215            PC56   
20            PC82            215            PC56   
21            PC83            309           PC143   
22            PC89            696            PC84   
23            PC89            696            PC85   
24            PC99            146            PC99   
25           PC106            145           PC151   
26           PC106            145           PC152   
27           PC106            145           PC162   
28           PC123             77            PC13   
29           PC123             77            PC17   
30           PC130           1917           PC207   
31           PC130           1917           PC229   
32           PC130           1917           PC249   
33           PC133           2056           PC115   
34           PC134            140           PC189   
35           PC134            140           PC204   
36           PC137           1097           PC129   
37           PC140            684            PC27   
38           PC141            684            PC27   
39           PC152            622           PC146   
40           PC162            942           PC231   
41           PC164            861           PC110   
42           PC167            861           PC110   
43           PC166             30            PC60   
44           PC168           1498           PC112   
45           PC171           2292           PC137   
46           PC174            778           PC153   
47           PC177            778           PC153   
48           PC176              9           PC168   
49           PC287              9           PC168   
50           PC185           1504            PC71   
51           PC186           1504            PC71   
52           PC240            302           PC145   
53           PC241            652           PC180   
54           PC247            652           PC180   
55           PC265            647            PC91   

                                   domain-description  
0                        CH; Calponin homology domain  
1                      Flp_C; Recombinase Flp protein  
2   AP_endonuc_2; Xylose isomerase-like, TIM barre...  
3                     BOP1NT; BOP1, N-terminal domain  
4   FDX-ACB; Ferrodoxin-fold anticodon-binding domain  
5     Cnd1_N; Condensin complex subunit 1, N-terminal  
6   AA_permease_2; Amino acid/polyamine transporter I  
7     Glycos_transf_1; Glycosyl transferase, family 1  
8    Diphthamide_syn; Diphthamide synthesis DPH1/DPH2  
9               ABC2_membrane; ABC-2 type transporter  
10                                          Trypsin_2  
11                                          Trypsin_2  
12               Cation_efflux; Cation efflux protein  
13  HCO3_cotransp; Bicarbonate transporter, C-term...  
14  HCO3_cotransp; Bicarbonate transporter, C-term...  
15  Candida_ALS_N; Agglutinin-like protein, N-term...  
16  Anth_synt_I_N; Anthranila

In [23]:
print('The number of shared domains between SL and nSL pairs,after PCA, is = ',len(df_shared_domains), 'out of', len(df_sl)+len(df_nsl), 'so the',100*len(df_shared_domains)/(len(df_sl)+len(df_nsl)),'%')

The number of shared domains between SL and nSL pairs,after PCA, is =  56 out of 568 so the 9.859154929577464 %


## Domains exclusive to SL

In [24]:
df_sl_exclusive=pd.merge(df_sl,df_nsl,how='left',on='domain-number')

domains_name=np.unique(data_domains['domain-name'])
domains_description=np.unique(data_domains['domain-descrip'])

df_sl_exclusive['domain-description']=domains_description[df_sl_exclusive['domain-number']]
df_sl_exclusive[pd.isnull(df_sl_exclusive['pca-component_y'])]

pca-component_x  domain-number pca-component_y  \
0               PC0            242             NaN   
1               PC1            191             NaN   
3               PC3             58             NaN   
4               PC4              3             NaN   
5               PC5            875             NaN   
..              ...            ...             ...   
293           PC284            529             NaN   
294           PC285             65             NaN   
295           PC286           1414             NaN   
297           PC288            543             NaN   
298           PC289           2476             NaN   

                                    domain-description  
0                  Atg31; Autophagy-related protein 31  
1                Alk_phosphatase; Alkaline phosphatase  
3                    AEP1; ATPase expression protein 1  
4    2-Hacid_dh_C; D-isomer specific 2-hydroxyacid ...  
5                                      FDF; FDF domain  
..                                                 ...  
293                                               CybS  
294  AIM3; Altered inheritance of mitochondria prot...  
295           Mad3_BUB1_I; Mad3/Bub1 homology region 1  
297              D123; Cell division cycle protein 123  
298                 Shugoshin_C; Shugoshin, C-terminal  

[243 rows x 4 columns]

In [25]:
print('The number of exclusive domains from SL pairs, after PCA, is = ',len(df_sl_exclusive), 'out of', len(df_sl)+len(df_nsl), 'so the',100*len(df_sl_exclusive)/(len(df_sl)+len(df_nsl)),'%')

The number of exclusive domains from SL pairs, after PCA, is =  299 out of 568 so the 52.640845070422536 %


In [26]:
df_nsl_exclusive=pd.merge(df_sl,df_nsl,how='right',on='domain-number')

domains_name=np.unique(data_domains['domain-name'])
domains_description=np.unique(data_domains['domain-descrip'])

df_nsl_exclusive['domain-description']=domains_description[df_nsl_exclusive['domain-number']]
df_nsl_exclusive[pd.isnull(df_nsl_exclusive['pca-component_x'])]

pca-component_x  domain-number pca-component_y  \
56              NaN           1028             PC0   
57              NaN            265             PC1   
58              NaN           2113             PC2   
59              NaN            987             PC4   
60              NaN              2             PC5   
..              ...            ...             ...   
280             NaN            318           PC270   
281             NaN           2382           PC273   
282             NaN           1912           PC274   
283             NaN           2137           PC275   
284             NaN              7           PC277   

                                    domain-description  
56                 GatB_Yqey; Asn/Gln amidotransferase  
57                            BCS1_N; BCS1, N-terminal  
58       Rav1p_C; RAVE complex protein Rav1 C-terminal  
59            GIT_SHD; GIT, Spa2 homology (SHD) domain  
60   2-Hacid_dh; D-isomer specific 2-hydroxyacid de...  
..                                                 ...  
280  CAP_C; Adenylate cyclase-associated CAP, C-ter...  
281  SRP19; Signal recognition particle, SRP19 subunit  
282  Phosphodiest; Type I phosphodiesterase/nucleot...  
283  Rib_5-P_isom_A; Ribose 5-phosphate isomerase, ...  
284       3-HAO; 3-hydroxyanthranilic acid dioxygenase  

[229 rows x 4 columns]

In [27]:
print('The number of exclusive domains from non SL pairs, after PCA, is = ',len(df_nsl_exclusive), 'out of', len(df_sl)+len(df_nsl), 'so the',100*len(df_nsl_exclusive)/(len(df_sl)+len(df_nsl)),'%')

The number of exclusive domains from non SL pairs, after PCA, is =  285 out of 568 so the 50.17605633802817 %


## Let see to the domains-number that get repeated after the PCA analysis , and selecting the most explanatory feature by its explained variance.

In [28]:
duplicated_features_sl=df_sl[df_sl.iloc[:,1].duplicated()]
repeated_features_sl=pd.DataFrame()

domains_name=np.unique(data_domains['domain-name'])
domains_description=np.unique(data_domains['domain-descrip'])

repeated_features_sl['domain-name']=domains_name[duplicated_features_sl.iloc[:,1]] 
repeated_features_sl['domain-description']=domains_description[duplicated_features_sl.iloc[:,1]]

duplicated_features_nsl=df_nsl[df_nsl.iloc[:,1].duplicated()]
repeated_features_nsl=pd.DataFrame()

repeated_features_nsl['domain-name']=domains_name[duplicated_features_nsl.iloc[:,1]] 
repeated_features_nsl['domain-description']=domains_description[duplicated_features_nsl.iloc[:,1]]

In [29]:
only_sl_pd=pd.merge(repeated_features_sl,repeated_features_nsl,how='left',on=['domain-name'])
data_only_sl = only_sl_pd[pd.isnull(only_sl_pd['domain-description_y'])]
data_only_sl['domain-description']=data_only_sl['domain-description_x']
data_only_sl.shape,df_sl_exclusive.shape

((54, 4), (299, 4))

In [30]:
only_nsl_pd=pd.merge(repeated_features_sl,repeated_features_nsl,how='right',on=['domain-name'])
data_only_nsl = only_nsl_pd[pd.isnull(only_nsl_pd['domain-description_x'])]
data_only_nsl

domain-name domain-description_x  \
0      PF00684                  NaN   
1      PF00169                  NaN   
2      PF04969                  NaN   
3      PF06733                  NaN   
4      PF06733                  NaN   
5      PF03372                  NaN   
6      PF02301                  NaN   
7      PF10373                  NaN   
8      PF04719                  NaN   
9      PF13921                  NaN   
10     PF08518                  NaN   
11     PF01644                  NaN   
12     PF09402                  NaN   
13     PF01663                  NaN   
14     PF11705                  NaN   
15     PF02466                  NaN   
16     PF09444                  NaN   
17     PF11496                  NaN   
18     PF13833                  NaN   
19     PF05625                  NaN   
20     PF12550                  NaN   
21     PF12550                  NaN   
22     PF00025                  NaN   
23     PF00025                  NaN   
24     PF00293                  NaN   
25     PF00293                  NaN   
26     PF11976                  NaN   
27     PF11976                  NaN   
28     PF02666                  NaN   
29     PF02666                  NaN   
30     PF12348                  NaN   
31     PF10471                  NaN   
32     PF08238                  NaN   
33     PF04048                  NaN   
34     PF00285                  NaN   
35     PF09497                  NaN   
36     PF08586                  NaN   
37     PF00635                  NaN   
38     PF01248                  NaN   
39     PF04667                  NaN   
40     PF01977                  NaN   
41     PF08505                  NaN   
42     PF08505                  NaN   
43     PF07823                  NaN   
44     PF15365                  NaN   
45     PF10375                  NaN   
46     PF10375                  NaN   
47     PF10375                  NaN   
48     PF08550                  NaN   
49     PF04147                  NaN   
50     PF09801                  NaN   
51     PF16755                  NaN   
52     PF01217                  NaN   
53     PF10609                  NaN   
54     PF00250                  NaN   
55     PF01302                  NaN   
56     PF01302                  NaN   
57     PF13519                  NaN   

                                 domain-description_y  
0   CDP-OH_P_transf; CDP-alcohol phosphatidyltrans...  
1        AMP-binding; AMP-dependent synthetase/ligase  
2   MetRS-N; Methionyl-tRNA synthetase, N-terminal...  
3     Ost5; Oligosaccharyltransferase complex subunit  
4     Ost5; Oligosaccharyltransferase complex subunit  
5   HCO3_cotransp; Bicarbonate transporter, C-term...  
6                        FAT; PIK-related kinase, FAT  
7                   Rsa3; Ribosome-assembly protein 3  
8           MT; Dynein heavy chain, coiled coil stalk  
9                                           Trypsin_2  
10  Polyketide_cyc; Coenzyme Q-binding protein COQ...  
11       DUF3402; Protein of unknown function DUF3402  
12                Rbsn; Rabenosyn, Rab binding domain  
13  DUF3543; Serine/threonine-protein kinase, C-te...  
14                              SSrecog; SSRP1 domain  
15      FTHFS; Formate-tetrahydrofolate ligase, FTHFS  
16           Rer1; Retrieval of early ER protein Rer1  
17                                    SPX; SPX domain  
18  Topoisom_bac; DNA topoisomerase, type IA, central  
19                                               NMD3  
20               Sortilin-Vps10; Sortilin, N-terminal  
21               Sortilin-Vps10; Sortilin, N-terminal  
22  5-FTHF_cyc-lig; 5-formyltetrahydrofolate cyclo...  
23  5-FTHF_cyc-lig; 5-formyltetrahydrofolate cyclo...  
24        A_deaminase; Adenosine/AMP deaminase domain  
25        A_deaminase; Adenosine/AMP deaminase domain  
26                                              Sec20  
27                                              Sec20  
28                                          Flocculin  
29                   

In [31]:
shared_domains_pd=pd.merge(repeated_features_sl,repeated_features_nsl,how='inner',on=['domain-name'])
shared_domains_pd


Empty DataFrame
Columns: [domain-name, domain-description_x, domain-description_y]
Index: []

In [32]:
a_sl=x_sl.iloc[:,np.sort(df_sl.iloc[:,1])]
a_sl.describe().loc['mean'].hist(bins=8),
a_sl.describe().loc['std'].hist(bins=8,alpha=0.4)
plt.xlim([0,0.2])

(0.0, 0.2)

In [33]:
a_nsl=x_nsl.iloc[:,np.sort(df_nsl.iloc[:,1])]
a_nsl.describe().loc['mean'].hist(bins=8),
a_nsl.describe().loc['std'].hist(bins=8,alpha=0.4)
plt.xlim([0,0.2])

(0.0, 0.2)

### Proof of concept with *BEM1*

- Gather the domains of SL and nSL pairs from BEM1
- Check if the domains for the SL and nSL pairs are inside the most important domains after PCA.

In [34]:
domains_bem1=data_domains[data_domains['name']=='BEM1']['domain-descrip']
sl_bem1=data_sl[data_sl['gene-query-name']=='BEM1']['gene-target-name']

In [35]:
sl_bem1

2846      BEM2
2847     CDC24
5004      MSB1
13777     SMI1
13778     BNI1
13779     BEM2
14385     SKN7
Name: gene-target-name, dtype: object

In [36]:
data_sl[data_sl['gene-query-name']=='BEM1']

gene-query gene-target gene-query-name gene-target-name  \
2846     YBR200W     YER155C            BEM1             BEM2   
2847     YBR200W     YAL041W            BEM1            CDC24   
5004     YBR200W     YOR188W            BEM1             MSB1   
13777    YBR200W     YGR229C            BEM1             SMI1   
13778    YBR200W     YNL271C            BEM1             BNI1   
13779    YBR200W     YER155C            BEM1             BEM2   
14385    YBR200W     YHR206W            BEM1             SKN7   

                                  gene-query-description  \
2846   SRO1|phosphatidylinositol-3-phosphate-binding ...   
2847   SRO1|phosphatidylinositol-3-phosphate-binding ...   
5004   SRO1|phosphatidylinositol-3-phosphate-binding ...   
13777  SRO1|phosphatidylinositol-3-phosphate-binding ...   
13778  SRO1|phosphatidylinositol-3-phosphate-binding ...   
13779  SRO1|phosphatidylinositol-3-phosphate-binding ...   
14385  SRO1|phosphatidylinositol-3-phosphate-binding ...   

                                 gene-target-description     interaction-type  \
2846                           IPL2|SUP9|TSL1|L000000168  Synthetic Lethality   
2847   CLS4|Rho family guanine nucleotide exchange fa...  Synthetic Lethality   
5004                                          L000001184  Synthetic Lethality   
13777                                    KNR4|L000000909  Synthetic Lethality   
13778                   PPF3|SHE5|formin BNI1|L000000190  Synthetic Lethality   
13779                          IPL2|SUP9|TSL1|L000000168  Synthetic Lethality   
14385  BRY1|POS9|kinase-regulated stress-responsive t...     Dosage Lethality   

            paper-source  
2846   Peterson J (1994)  
2847   Peterson J (1994)  
5004     Bender A (1991)  
13777   Gorelik M (2011)  
13778   Gorelik M (2011)  
13779   Gorelik M (2011)  
14385   Bouquin N (1999)